In this notebook, we will create and compare several models. Our target feature for this notebooks is (Likes+Comments/Views)
which we will call "engagement". 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"all_features_final.csv")
features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref", "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products", "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]
df = df[features]